# Summarize large text

Anton Antonov   
August 2025

----

## Setup

In [1]:
use LLM::Graph;
use Graph;
use Data::Importers;

In [2]:
my $engine = 'neato';

neato

----

## LLM graph

In [3]:
my %rules =
TypeOfInput => sub ($_) {"Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ llm-prompt('NothingElse')('single string')},

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, 'HTML') },

MindMap => sub ($IngestText) { 
    [
        "Create a concise mermaid-js mind-map for the text:",
        $IngestText,
        "All node messages should be in quotes.",
        llm-prompt("NothingElse")("correct mermaid-js")
    ]
},

Report => { eval-function => 
    sub ($Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            '# Report',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
}
;

{IngestText => {eval-function => sub { }}, MindMap => sub { }, Report => {eval-function => sub { }}, Summary => sub { }, ThinkingHats => sub { }, TopicsTable => sub { }, TypeOfInput => sub { }}

Make the graph:

In [4]:
my $gCombinedSummary = LLM::Graph.new(%rules)

LLM::Graph(size => 7, nodes => IngestText, MindMap, Report, Summary, ThinkingHats, TopicsTable, TypeOfInput)

----

## Full computation

URL in text statistics:

In [5]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-3-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5861 words => 1158 lines => 156)

Computation:

In [6]:
$gCombinedSummary.eval($url)

LLM::Graph(size => 7, nodes => IngestText, MindMap, Report, Summary, ThinkingHats, TopicsTable, TypeOfInput)

Show the corresponding graph:

In [7]:
#% html
$gCombinedSummary.graph.dot(engine => 'dot', node-shape => 'ellipse', node-width => 1.2 ):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText

Save the document:

In [8]:
spurt('./Report.md', $gCombinedSummary.rules<Report><result>);
shell "open ./Report.md"

Proc.new(in => IO::Pipe, out => IO::Pipe, err => IO::Pipe, os-error => Str, exitcode => 0, signal => 0, pid => 2887, command => ("open ./Report.md",))

Final result:

In [9]:
#% markdown
$gCombinedSummary.rules<Report><result>.subst(/'```html' | '```' $/):g

# Report

## Summary

The presentation titled "Graph Neat Examples in Raku" by Anton Antonov demonstrates the functionalities of the Raku package "graph" through nested graphs and file system graphs, visualized using JavaScript libraries d3.js and fre GS. Nested graphs involve applying functions repeatedly to generate symbolic representations, while file system graphs visualize directory structures and their files, including examples from Mathematica notebooks and the Raku doc repository. The talk highlights methods for creating, analyzing, and displaying these graphs, such as computing vertex degrees and using various graph embeddings and layouts.

## Topics

<table border="1"><thead><tr><th>theme</th><th>content</th></tr></thead><tbody><tr><td align=left>Introduction and Presentation Overview</td><td align=left>Introduction of presenter, date, and overview of nested graphs and file system graph examples.</td></tr><tr><td align=left>Definition and Purpose of Neat Examples</td><td align=left>Explanation of neat examples as concise code producing compelling visual textual outputs.</td></tr><tr><td align=left>Graph Package and Visualization Tools</td><td align=left>Description of Raku graph package and visualization using JavaScript libraries d3js and fre GS.</td></tr><tr><td align=left>Nested Graphs Basic Examples</td><td align=left>Demonstrates simple nested graphs with function applications and symbolic representation.</td></tr><tr><td align=left>Nested Graphs with Two Functions</td><td align=left>Shows nested graphs using two functions and visualizing the results with JavaScript and SVG.</td></tr><tr><td align=left>Modulo Graph Example</td><td align=left>Explanation of more complex graph using modulo operations on vertex values producing directed graphs.</td></tr><tr><td align=left>Range Graph Example</td><td align=left>Using range graphs to demonstrate nesting steps and allomorph applications producing larger graphs.</td></tr><tr><td align=left>Graph Properties and Visualization</td><td align=left>Describes computing in-degree and out-degree, directed vs undirected graphs, and visualization challenges.</td></tr><tr><td align=left>File System Graphs Introduction</td><td align=left>Introduction to file system graphs using the Puffs package and Mathematica notebooks directory.</td></tr><tr><td align=left>Constructing File System Graphs</td><td align=left>Details on creating edges between directories and files for visualization.</td></tr><tr><td align=left>File System Graph Visualization</td><td align=left>Use of circular embedding and displaying file names like CSV, zip, and Mathematica notebooks.</td></tr><tr><td align=left>Raku Documentation File System Graph</td><td align=left>Cloning Raku doc repo, plotting its file system graph with fre GS and force display.</td></tr><tr><td align=left>Interactivity and Node Selection in Graphs</td><td align=left>Discussion on node selection difficulties and attempts to improve graph readability.</td></tr><tr><td align=left>Conclusion</td><td align=left>Final remarks and closing of the presentation.</td></tr></tbody></table>

## Mind map

```mermaid
mindmap
  root(( "Graph Neat Presentation" ))
    subgraph "Introduction"
      A1("Title: Graph Neat")
      A2("Presenter: Anton Antonov")
      A3("Date: November 28, 2024")
      A4("Focus: Neat examples in Raku set")
    end
    subgraph "Examples Sets"
      B1("Nested Graphs")
      B2("File System Graphs")
    end
    subgraph "Neat Examples Definition"
      C1("Concise, straightforward code")
      C2("Compelling visual & textual outputs")
    end
    subgraph "Tools & Libraries"
      D1("Raku package: graph")
      D2("Visualization: JavaScript & D3.js")
      D3("SVG images via GraphViz framework")
    end
    subgraph "Nested Graphs"
      E1("Concept: applying function(s) to initial value")
      E2("Class: graph::nested with new signature")
      E3("Example 1: one function f applied multiple times")
      E4("Example 2: two functions f and g applied")
      E5("Display: SVG generated & rendered in Jupyter notebook")
      E6("Mod Graph")
      E7("Range Graph")
      E8("Degree computations (in-degree/out-degree)")
      E9("Visualization using fre GS (busy graph)")
    end
    subgraph "Mod Graph Details"
      F1("Operation: square, add one, mod 10")
      F2("Vertices: integers 0-9")
      F3("Result: directed graph")
    end
    subgraph "Range Graph Details"
      G1("Start: value '9' as string")
      G2("One step nesting & second step nesting")
      G3("Directed and undirected graphs")
      G4("Complete graph when undirected")
      G5("Degree methods with abbreviation P")
    end
    subgraph "File System Graphs"
      H1("Package: Puffs")
      H2("Data: Mathematica notebooks directory")
      H3("Process: split files & directories")
      H4("Edges: root to subdirectories to files")
      H5("Files count: 186")
      H6("Visualization: circular embedding")
      H7("Showing filenames: csv, zip, nb files")
    end
    subgraph "Raku Doc Files Graph"
      I1("Source: Raku doc GitHub repository")
      I2("Visualization: JavaScript & fre GS")
      I3("Display: force layout, squashed graph")
      I4("Nodes: recognizable Raku components")
      I5("Nodes interactable for better legibility")
    end
    subgraph "Conclusion"
      J1("Final neat example shown")
      J2("Thank you")
    end
```

## Thinking hats

<table>
  <tr>
    <th>Hat Name</th>
    <th>Perspective</th>
    <th>Feedback</th>
    <th>New Ideas</th>
  </tr>
  <tr>
    <td>White Hat</td>
    <td>Information and Facts</td>
    <td>The presentation covers the use of the Raku language with the Graph package to generate and visualize nested and file system graphs. It utilizes JavaScript libraries such as D3.js and Freggs for rendering SVG graphs, and references the GraphVis framework for graph specification. Examples include nested function compositions, modular arithmetic graphs, range graphs, and directory structure visualizations. The material is technical and assumes familiarity with graph theory concepts and Raku programming.</td>
    <td>Include performance benchmarks of graph generation and rendering, provide a detailed explanation of the graph data structures used in Raku, add comparative analysis with other graph visualization tools.</td>
  </tr>
  <tr>
    <td>Black Hat</td>
    <td>Judgement and Caution</td>
    <td>The presentation may be too technical and dense for audiences unfamiliar with Raku or graph theory, risking disengagement. Visualization of complex graphs appears cluttered and difficult to interpret, especially with large datasets like filesystem graphs. The dependence on JavaScript libraries in a Jupyter notebook environment could introduce compatibility or rendering issues. The old graph Vis framework might have limitations or lack modern features. Also, the code snippets and explanations sometimes lack clarity and fluidity, reducing accessibility.</td>
    <td>Focus on simplifying examples for clarity, improve graph layout algorithms to reduce clutter, consider alternative or updated visualization frameworks, provide clearer step-by-step walk-throughs for code segments.</td>
  </tr>
  <tr>
    <td>Gray Hat</td>
    <td>Cynicism and Skepticism</td>
    <td>The presentation seems to overemphasize the novelty of using Raku with these graph tools when other languages and frameworks have more mature ecosystems. The choice of an "old" graph visualization framework suggests a reluctance or inability to adopt more modern or efficient solutions, which may be due to inertia or lack of resources. The complexity and verbosity might serve more as an intellectual display than practical utility. Also, the "neat" label applied to examples feels like marketing jargon to gloss over usability issues.</td>
    <td>Question why Raku is chosen over more mainstream languages, explore motivations behind sticking with older frameworks, investigate if the complexity provides real user value or is just academic, consider more pragmatic or streamlined approaches.</td>
  </tr>
  <tr>
    <td>Red Hat</td>
    <td>Feelings and Emotion</td>
    <td>The presentation evokes curiosity and excitement about visualizing abstract concepts like nested function graphs and file structures. However, there is some frustration due to the cluttered visual output and occasional unclear explanations. The passion behind the work is evident, which can inspire interest among Raku users. The live coding and dynamic rendering in Jupyter notebooks add an engaging and interactive feel, though it might overwhelm some viewers.</td>
    <td>Incorporate more user-friendly visuals and interaction to reduce frustration, use storytelling to connect emotionally with audience, add motivational context about why these visualizations matter.</td>
  </tr>
  <tr>
    <td>Yellow Hat</td>
    <td>Benefits and Creativity</td>
    <td>This approach demonstrates powerful capabilities of combining Raku's functional programming with graph visualization, inspiring new ways to understand data structures and code behavior. The use of nested graphs can help users intuitively grasp function compositions. File system graphs can aid in codebase exploration, documentation, and debugging. Integrating JavaScript visualization libraries opens creative opportunities for interactive and dynamic outputs within notebooks.</td>
    <td>Develop interactive graph exploration tools, extend to real-time data visualization from live Raku programs, create educational modules teaching graph theory through these examples.</td>
  </tr>
  <tr>
    <td>Green Hat</td>
    <td>Profitability and Potential</td>
    <td>There is potential for monetization through developing and selling advanced Raku graph visualization libraries or tools tailored for developers and data scientists. The niche nature of Raku may limit market size, but targeting educational institutions or enterprises needing custom graph analysis tools could be profitable. Offering consulting or workshops on using these graph tools in Raku projects is another avenue. Packaging these visualizations into user-friendly dashboards may increase commercial appeal.</td>
    <td>Create a commercial plugin or GUI tool for Raku graph visualization, offer training sessions for companies adopting Raku, build SaaS platforms for graph analytics using this technology.</td>
  </tr>
  <tr>
    <td>Blue Hat</td>
    <td>Usefulness and Opportunities</td>
    <td>The presentation highlights useful applications such as visualizing nested computations and file system hierarchies which can simplify complex data understanding. Opportunities exist to integrate these visualizations into Raku IDEs or developer tools to improve productivity. Extending support to other graph types or datasets (like social networks or dependency graphs) could expand usefulness. There is room to refine UI/UX for better graph interaction and clarity.</td>
    <td>Develop plugins for Raku IDEs to visualize code dependencies, extend graph visualizations to other domains, improve graph interaction (zoom, filter, search).</td>
  </tr>
  <tr>
    <td>Purple Hat</td>
    <td>Assessment and Direction</td>
    <td>The idea of leveraging Raku's Graph package with JavaScript visualization libraries to demonstrate nested and file system graphs is technically sound and has promising educational and practical value. However, challenges with clarity, visual clutter, and possibly outdated frameworks should be addressed. There is untapped potential in making these tools more accessible, interactive, and commercially viable. Balancing technical depth with user-friendliness will be crucial to broaden impact and adoption.</td>
    <td>Focus on simplifying and clarifying visualizations, update or replace legacy visualization frameworks, explore commercial and educational applications, and invest in UI/UX improvements to maximize reach and utility.</td>
  </tr>
</table>

----

## Partial evaluation

Clone:

In [10]:
# my $gCombinedSummary2 = $gCombinedSummary.clone;

()

In [11]:
# $gCombinedSummary2.eval(
#     $url,
#     Summary => $gCombinedSummary.rules<Summary><result>,
#     TopicsTable => $gCombinedSummary.rules<TopicsTable><result>
# )

()